# $group 的操作

In [ ]:
# grouping by year and getting a count per year using the { $sum: 1 } pattern
db.movies.aggregate([
    {
        $group: {
            _id: "$year",
            "numFilmsThisYear": { $sum: 1 }
        }
    }
]).pretty()

In [ ]:
# grouping as before, then sorting in descending order based on the count
db.movies.aggregate([
    {
        $group: {
            _id: "$year",
            "count": { $sum: 1 }
        }
    },
    { $sort: { count: -1 }}
]).pretty()

In [ ]:
# grouping on the number of directors a film has, demonstrating that we have to
# validate types to protect some expressions
db.movies.aggregate([
    {
        $group: {
            _id: { "numDirectors": { $cond: [ { $isArray: "$directors" }, { $size: "$directors" }, 0 ] } },
            numFilms: { $sum: 1},
            "averageMetacritic": { $avg: "$metacritic" }
        }
    },
    { $sort: { "_id.numDirectors": -1 }}
]).pretty()

In [ ]:
# showing how to group all documents together. By convention, we use null or an empty string, ""
db.movies.aggregate([
    {
        $group: {
            _id: null,
            count: { $sum: 1 }
        }
    }
])

In [ ]:
# filtering results to only get documents with a numeric metacritic value
db.movies.aggregate([
    {
        $match: {
            metacritic: { $gte: 0 }
        }
    },
    {
        $group: {
            _id: null,
            averageMetacritic: { $avg: "$metacritic" }
        }
    }
]).pretty()

# Accumulator Expressions: \\$sum \\$avg \\$max \\$min \\$stdDevPop \\$stdDevSamp 的操作

In [ ]:
# using $reduce to get the highest temperature
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            max_high: {
                $reduce: {
                    input: "$trends",
                    initialValue: -Infinity,
                    in: { $cond: [ { $gt: [ "$$this.avg_high_tmp", "$$value" ]}, "$$this.avg_high_tmp", "$$value" ]}
                }
            }
        }
    }
])

In [ ]:
# performing the inverse, grabbing the lowest temperature
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            min_high: {
                $reduce: {
                    input: "$trends",
                    initialValue: Infinity,
                    in: { $cond: [ { $lt: [ "$$this.avg_low_tmp", "$$value" ]}, "$$this.avg_low_tmp", "$$value" ]}
                }
            }
        }
    }
])

In [ ]:
# note that these two operations can be done with the following operations can
# be done more simply. The following two expressions are functionally identical
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            max_high: { $max: "$trends.avg_high_tmp" }
        }
    }
])

db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            min_high: { $min: "$trends.avg_low_tmp" }
        }
    }
])

In [ ]:
# getting the average and standard deviations of the consumer price index
db.icecream_data.aggregate([
    {
        $project: {
            _id: 0,
            average_cpi: { $avg: "$trends.icecream_cpi"},
            cpi_deviation: { $stdDevPop: "$trends.icecream_cpi" }
        }
    }
])

In [ ]:
# using the $sum expression to get total yearly sales
db.icecream_data.aggregate([
    {
      $project: {
        _id: 0,
        "yearly_sales (millions)": { $sum: "$trends.icecream_sales_in_millions" }
      }
    }
  ])

### 範例01

In [ ]:
db.movies.aggregate([
    {
        $match: {
            "imdb.votes": { $gte: 1}
        }
    },
    {
        $group: {
            _id: null,
            maxVotes: { $max: "$imdb.votes" },
            minVotes: { $min: "$imdb.votes" }
        }
    }
])

### 範例02

In [ ]:
db.grades.aggregate([
    {
        $project: {
            maxGrade: { $max: "$quizzes" }
        }
    }
])

### 範例03

In [ ]:
# 參考01
db.movies.aggregate([
    {
        $match: { 
            awards: { $exists: true }
        }
    },
    {
        $project: {
            _id: 0,
            title: 1,
            awards: { $split: [ "$awards", " " ]},
            imdb_rating: "$imdb.rating" 
        }
    },
    {
        $match: {

            "awards.0": "Won",
            "awards.2": { $in: [ "Oscar.", "Oscars." ]}
        }
    },
    {
        $group: {
            _id: null,
            highest_rating: { $max: "$imdb_rating" },
            lowest_rating: { $min: "$imdb_rating" },
            average_rating: { $avg: "$imdb_rating" },
            deviation: { $stdDevSamp: "$imdb_rating" }
        }
    }
]).pretty()

In [ ]:
# 參考02
db.movies.aggregate([
  {
    $match: {
      awards: /Won \d{1,2} Oscars?/
    }
  },
  {
    $group: {
      _id: null,
      highest_rating: { $max: "$imdb.rating" },
      lowest_rating: { $min: "$imdb.rating" },
      average_rating: { $avg: "$imdb.rating" },
      deviation: { $stdDevSamp: "$imdb.rating" }
    }
  }
])

# $unwind 的操作

In [ ]:
# finding the top rated genres per year from 2010 to 2015...
db.movies.aggregate([
    {
        $match: {
            "imdb.rating": { $gt: 0},
            year: { $gte: 2010, $lte: 2015},
            runtime: { $gte: 90 }
        }
    },
    { $unwind: "$genres" },
    {
        $group: {
            _id: { year: "$year", genre: "$genres" },
            average_rating: { $avg: "$imdb.rating" }
        }
    },
    { $sort: { "_id.year": -1, average_rating: -1 }}

]).pretty()

In [ ]:
# unfortunately we got too many results per year back. Rather than peform some
# other complex grouping and matching, we just append a simple group and sort
# stage, taking advantage of the fact the documents are in the order we want
db.movies.aggregate([
    {
        $match: {
            "imdb.rating": { $gt: 0 },
            year: { $gte: 2010, $lte: 2015 },
            runtime: { $gte: 90 }
        }
    },
    { $unwind: "$genres" },
    {
        $group: {
            _id: { year: "$year", genre: "$genres" },
            average_rating: { $avg: "$imdb.rating" }
        }
    },
    { $sort: { "_id.year": -1, average_rating: -1 }},
    {
        $group: {
            _id: "$_id.year",
            genre: { $first: "$_id.genre" },
            average_rating: { $first: "$average_rating" }
        }
    },
    { $sort: { _id: -1 }}
    
]).pretty()

### 範例01

In [ ]:
# 參考01
db.movies.aggregate([
    {
        $match: {
            languages: { $in: [ "English" ]}
        }
    },
    { $unwind: "$cast"},
    {
        $group: {
            _id: "$cast",
            numFilms: { $sum: 1 },
            average: { $avg: "$imdb.rating" }
        }
    },
    {
        $project: {
            _id: 1,
            numFilms: 1,
            average: { $trunc: [ "$average", 1 ]}
        }
    },
    { $sort: { numFilms: -1 }},
    { $limit: 1 }
])

In [ ]:
# 參考02
db.movies.aggregate([
  {
    $match: {
      languages: "English"
    }
  },
  {
    $project: { _id: 0, cast: 1, "imdb.rating": 1 }
  },
  {
    $unwind: "$cast"
  },
  {
    $group: {
      _id: "$cast",
      numFilms: { $sum: 1 },
      average: { $avg: "$imdb.rating" }
    }
  },
  {
    $project: {
      numFilms: 1,
      average: {
        $divide: [{ $trunc: { $multiply: ["$average", 10] } }, 10]
      }
    }
  },
  {
    $sort: { numFilms: -1 }
  },
  {
    $limit: 1
  }
])

# $lookup 的操作

In [ ]:
# familiarizing with the air_alliances schema
db.air_alliances.findOne()

# familiarizing with the air_airlines schema
db.air_airlines.findOne()

# performing a lookup, joining air_alliances with air_airlines and replacing
# the current airlines information with the new values
db.air_alliances.aggregate([
    {
        $lookup: {
            from: "air_airlines",
            localField: "airlines",
            foreignField: "name",
            as: "airlines"
        }
    }
]).pretty()

In [ ]:
db.air_airlines.aggregate([
    {
        $lookup: {
            from: "air_alliances",
            localField: "name",
            foreignField: "airlines",
            as: "airline"
        }
    },
    {
        $match: {
            airline: { $elemMatch: { $exists: true }}
        }
    }
]).pretty()

### 範例01

In [ ]:
db.air_alliances.findOne()
db.air_routes.findOne()

# 注意要記得 $unwind 這一個 stage 
db.air_routes.aggregate([
    {
        $match: {
            airplane: /747|380/
        }
    },
    {
        $lookup: {
            from: "air_alliances",
            localField: "airline.name",
            foreignField: "airlines",
            as: "alliance"
        }
    },
    { $unwind: "$alliance" },
    {
        $group: {
            _id: { alliance: "$alliance.name" },
            num: { $sum: 1 }
        }
    },
    { $sort: { num: -1 }}
]).pretty()

### 範例02

In [ ]:
# 參考01
db.air_routes.aggregate([
    {
        $match: {
            $or: [ { src_airport: "JFK", dst_airport: "LHR" }, { src_airport: "LHR", dst_airport: "JFK" }]
        }
    },
    {
        $lookup: {
            from: "air_alliances",
            localField: "airline.name",
            foreignField: "airlines",
            as: "air_alliance"
        }
    },
    {
        $match: { air_alliance: { $ne: [] }}
    },
    {
        $group: {
            _id: "$air_alliance.name",
            airlines: { $addToSet: "$airline.name"}
        }
    },
    {
      $addFields: { airlinesNum: { $size: "$airlines" }}   
    },
    {
        $sort: { airlines: -1 }
    }
]).pretty()

In [ ]:
# 參考02
db.air_routes.aggregate([
  {
    $match: {
      src_airport: { $in: ["LHR", "JFK"] },
      dst_airport: { $in: ["LHR", "JFK"] }
    }
  },
  {
    $lookup: {
      from: "air_alliances",
      foreignField: "airlines",
      localField: "airline.name",
      as: "alliance"
    }
  },
  {
    $match: { alliance: { $ne: [] } }
  },
  {
    $addFields: {
      alliance: { $arrayElemAt: ["$alliance.name", 0] }
    }
  },
  {
    $group: {
      _id: "$airline.id",
      alliance: { $first: "$alliance" }
    }
  },
  {
    $sortByCount: "$alliance"
  }
])

# $graphLookup 的操作

In [ ]:
db.parent_reference.find()

db.parent_reference.find({ name: "Dev" })

db.parent_reference.find({ reports_to: 1 })

In [ ]:
# 查找下屬
db.parent_reference.aggregate([
    { $match: { name: "Eliot" }},
    {
        $graphLookup: {
            from: "parent_reference",
            startWith: "$_id",
            connectFromField: "_id",
            connectToField: "reports_to",
            as: "all_reports"
        }
    }
]).pretty()

In [ ]:
# 查找上屬
db.parent_reference.aggregate([
    { $match: { name: "Shannon" }},
    {
        $graphLookup: {
            from: "parent_reference",
            startWith: "$reports_to",
            connectFromField: "reports_to",
            connectToField: "_id",
            as: "bosses"
        }
    }
]).pretty()

In [ ]:
db.child_reference.find()

db.child_reference.findOne({ name: "Dev"})

In [ ]:
# 查找下屬
db.child_reference.aggregate([
    { $match: { name: "Dev" }},
    {
        $graphLookup: {
            from: "child_reference",
            startWith: "$direct_reports",
            connectFromField: "direct_reports",
            connectToField: "name",
            as: "all_reports"
        }
    }
]).pretty()

In [ ]:
# 查找下屬，只找 2 層
db.child_reference.aggregate([
    { $match: { name: "Dev" }},
    {
        $graphLookup: {
            from: "child_reference",
            startWith: "$direct_reports",
            connectFromField: "direct_reports",
            connectToField: "name",
            as: "till_2_level_reports",
            maxDepth: 1
        }
    }
]).pretty()

In [ ]:
db.child_reference.aggregate([
    { $match: { name: "Dev" }},
    {
        $graphLookup: {
            from: "child_reference",
            startWith: "$direct_reports",
            connectFromField: "direct_reports",
            connectToField: "name",
            as: "till_2_level_reports",
            maxDepth: 1,
            depthField: "level"
        }
    }
]).pretty()

In [ ]:
db.air_airlines.findOne()

db.air_routes.findOne()

In [ ]:
db.air_airlines.aggregate([
    { $match: { name: "TAP Portugal" }},
    {
        $graphLookup: {
            from: "air_routes",
            as: "chain",
            startWith: "$base",
            connectFromField: "dst_airport",
            connectToField: "src_airport",
            maxDepth: 0
            
        }
    }
    
]).pretty()

In [ ]:
db.air_airlines.aggregate([
    { $match: { name: "TAP Portugal" }},
    {
        $graphLookup: {
            from: "air_routes",
            as: "chain",
            startWith: "$base",
            connectFromField: "dst_airport",
            connectToField: "src_airport",
            maxDepth: 1
            
        }
    }
    
]).pretty()

In [ ]:
db.air_airlines.aggregate([
    { $match: { name: "TAP Portugal" }},
    {
        $graphLookup: {
            from: "air_routes",
            as: "chain",
            startWith: "$base",
            connectFromField: "dst_airport",
            connectToField: "src_airport",
            maxDepth: 1,
            restrictSearchWithMatch: { "airline.name": "TAP Portugal"}
            
        }
    }
    
]).pretty()